# David Fazylov- SQL Noir: Seven Mysteries (SQL Server)

_Detective Noir Theme — presentation-ready notebook_

This notebook contains **six** mysteries you can run against **Northwinds2022** (and one small custom _Mystery_ database for Case #1) for a total of 7 Mystery's  
All SQL is written for **Microsoft SQL Server**.

* * *

## How to use this notebook
- Each case begins with an **intro narrative** and **objectives**.
- Steps are presented as **short SQL blocks** you can run in SSMS, Azure Data Studio, or DBeaver.
- Replace schema/table names if your environment differs.
- Clear **BEGIN/END** markers surround each mystery.

---

# 🔎 MYSTERY #1 — The Missing MacBook at Queens College Library  
**Database:** `Mystery` (custom)  
**Theme:** Stolen laptop, a witness, and a tattooed right hand.

**Begin Case #1**

### Case Brief
A MacBook Pro vanished from the Queens College Library — 3rd Floor Study Area at 2:45 PM.  
A witness saw someone in a **black hoodie**, fleeing, with a **tattoo on the right hand**.

**Goal:** use basic SQL to locate the crime scene event, shortlist suspects, verify via interview, and find corroborating evidence.

### Tables (custom)
- `crime_scene(id, date, type, location, description)`  
- `suspects(id, name, attire, scar)`  
- `interviews(suspect_id, transcript)`  
- `evidence(detail)`

### Step 1 — Crime scene filter- We want only the **theft reports** in the **QC library --** the moment it all started

```sql
USE Mystery;
GO

SELECT type, location, description
FROM dbo.crime_scene
WHERE type = 'theft'
  AND location = 'Queens College Library';
```

### Step 2 — Narrow suspects by description - Now we look for **people who fit witness description** clothing can be changed! but Tattoos? they can not be hidden!!

```sql
SELECT id, name, attire, scar
FROM dbo.suspects
WHERE attire LIKE '%black hoodie%'
  AND scar   LIKE '%right hand%';
```

### Step 3 — Verify by interview transcript - this reduces the noise, from campus crowds and late night stragglers..

**down to just a few names now...**  the list isnt very long, its making things interesting.

```sql
SELECT s.id, s.name, i.transcript
FROM dbo.suspects AS s
JOIN dbo.interviews AS i
  ON i.suspect_id = s.id
WHERE s.id = 10;   -- replace with the candidate from Step 2
```

### Step 4 — Corroborating evidence - suspects talk and when they do? we listen!

```sql
SELECT *
FROM dbo.evidence
WHERE detail LIKE '%locker%'
   OR detail LIKE '%camera%'
   OR detail LIKE '%black hoodie%'
   OR detail LIKE '%tattoo%';
```

**Result:** our working suspect (e.g., *Alex Moreno*) matches the description and transcript, and the evidence supports the timeline.

**End Case #1**
---

# 🔎 MYSTERY #2 — The Case of the Vanishing Bulk Discounts  
**Database:** `Northwinds2022`  
**Theme:** Bulk orders (≥ 50 units) with **no discount** applied.

**Begin Case #2**

### Case Brief
Accounting rumors say some large orders never received their promised discounts.  
We’ll verify by finding orders whose **total quantity ≥ 50** and **zero discounted lines**.

### Step 1 —Lets survey the ledger. We begin with a look at the order system. No assumptions yet. Only **Structure,shape and volume.** We are hunting down the patterns where the rows show quantities , dates, and discount percentages this is where our ghost may be hiding.

```sql
USE Northwinds2022;
GO

SELECT TOP (5) * FROM Sales.[Order]      ORDER BY OrderId;
SELECT TOP (5) * FROM Sales.OrderDetail  ORDER BY OrderId, ProductId;
```

### Step 2 — Join orders to details to reconstruct the transaction trail. We connect orders to their line by line details because corruption usually happens where its not obvious, in the fine print! now we can form a picture. Which orders were big and which ones had big discounts? we are seeing where a problem can live.

```sql
SELECT
    o.OrderId,
    o.OrderDate,
    od.ProductId,
    od.Quantity,
    od.DiscountPercentage
FROM Sales.[Order] AS o
JOIN Sales.OrderDetail AS od
  ON od.OrderId = o.OrderId
ORDER BY o.OrderId, od.ProductId;
```

### Step 3 — Keep only bulk orders which are 50+ total items. thats our threshold of where discoutns must kick in!

so these orders qualify for discounts they should have them.. but do they?

```sql
SELECT
    o.OrderId,
    o.OrderDate,
    SUM(od.Quantity) AS TotalQuantity
FROM Sales.[Order] AS o
JOIN Sales.OrderDetail AS od
  ON od.OrderId = o.OrderId
GROUP BY o.OrderId, o.OrderDate
HAVING SUM(od.Quantity) >= 50
ORDER BY TotalQuantity DESC, o.OrderDate, o.OrderId;
```

### Step 4 — Bulk with zero discount lines (the anomaly) - if someone wanted to quietly benefit a buyer.. or themselves.. this is where they would hide it! this query only does 1 thing: it finds the bulk orders. if this returns even 1 record? then someone broke the rules!

```sql
SELECT
    o.OrderId,
    o.OrderDate,
    SUM(od.Quantity) AS TotalQuantity
FROM Sales.[Order] AS o
JOIN Sales.OrderDetail AS od
  ON od.OrderId = o.OrderId
GROUP BY o.OrderId, o.OrderDate
HAVING
    SUM(od.Quantity) >= 50
    AND SUM(CASE WHEN COALESCE(od.DiscountPercentage, 0) > 0 THEN 1 ELSE 0 END) = 0
ORDER BY TotalQuantity DESC, o.OrderDate, o.OrderId;
```

## **End Case #2 we found no one broke the rules, thats a relief!**

#   

# 🔎 MYSTERY #3 — Invisible Inventory (Products Never Ordered)

**Database:** `Northwinds2022`  
**Theme:** Products that **never** appeared in any order.

Case Brief: In the warehouse, dust settles on untouched boxes. But dust isnt tracked in the accounting logs. Inverntory numbers say everything is **active** and that everything is **selling**  but someone noticed something strange : Some products have never been ordered. How can that possibly happen at Northwinds? every prodcut is supposed to a part of the machine. If a product is sitting around? **someone ordered items that we shouldnt even be carrying!! we will find the culprit.**

**Begin Case #3**

### Step 1 — Sanity check, lets take a look at our product catalog and list everything we sell. This shows everything in our system , whether it sells or not. From names to IDs and descriptions.

```sql
SELECT TOP (5) * FROM Production.Product;
```

### Step 2 — Find what has been ordered and lets see the trail of real world demand so if a product appears here, cthen customers actually bought it and if it doesn't? its a ghost product.

```

-- Set B: ProductIds that appeared in orders
SELECT DISTINCT ProductId
FROM Sales.OrderDetail;

```

### Step 3 — Products with **no** orders (EXCEPT)

lets use a set difference to see what exists and what was ever used. The list is all products that exist but never appear in a single order

```sql
SELECT ProductId
FROM Production.Product
EXCEPT
SELECT DISTINCT ProductId
FROM Sales.OrderDetail;
```

SELECT 'No missing products — every product has sales history!' AS Finding;

If the result is empty: **all catalog products have sales history** → good inventory utilization.  
If there are results: these SKUs are **invisible inventory** and should be reviewed and we shoudl find those ghost products.

## **End Case #3 We have somved out mystery. no ghost products exist! great news. although we didn't reveal a culprit we revealed integrity exists even though suspicion lived! sometimes finding nothing wrong is a great thing.**

# 🔎 MYSTERY #4 — Phantom Big Spenders (Large Order Investigation)

**Database:** `Northwinds2022`  
**Theme:** Orders whose **value ≥ $5,000**.

CASE BRIEF: Some orders are so large that they **distort revenue!** Finance calls them **outliers** but we call them **leads.** we are here to find our largest spenders.. too large.. suspicious. Are they real VIP customers or is someone hiding **fraud** and hiking up our revenues artificially?

**Begin Case #4**

### Step 1 — Join orders to details

lets find what our system considers an order and its details. Just observe.

```sql
SELECT
    o.OrderId,
    o.OrderDate,
    od.ProductId,
    od.Quantity,
    od.UnitPrice,
    od.DiscountPercentage
FROM Sales.[Order] AS o
JOIN Sales.OrderDetail AS od
  ON od.OrderId = o.OrderId
ORDER BY o.OrderId, od.ProductId;
```

### Step 2 — Summarize order total- lets now compute each orders true value with any discounts.

```sql
SELECT
    o.OrderId,
    o.OrderDate,
    SUM(od.Quantity) AS TotalQuantity,
    COUNT(*) AS LineCount,
    SUM(od.UnitPrice * od.Quantity * (1 - COALESCE(od.DiscountPercentage, 0))) AS OrderValue
FROM Sales.[Order] AS o
JOIN Sales.OrderDetail AS od
  ON od.OrderId = o.OrderId
GROUP BY o.OrderId, o.OrderDate
ORDER BY OrderValue DESC;
```

### Step 3 — Flag big spenders (≥ 5000)- now we are finding some suspects and are isolating our whale customers and if this list comes back long? something will seem fishy, soemthing systemic may be happening. If the list returns short? lets pay attention  to who placed them and when

```sql
SELECT
    o.OrderId,
    o.OrderDate,
    SUM(od.Quantity) AS TotalQuantity,
    COUNT(*) AS LineCount,
    SUM(od.UnitPrice * od.Quantity * (1 - COALESCE(od.DiscountPercentage, 0))) AS OrderValue
FROM Sales.[Order] AS o
JOIN Sales.OrderDetail AS od
  ON od.OrderId = o.OrderId
GROUP BY o.OrderId, o.OrderDate
HAVING SUM(od.UnitPrice * od.Quantity * (1 - COALESCE(od.DiscountPercentage, 0))) >= 5000
ORDER BY OrderValue DESC;
```

## **End Case #4 we have found some fishy players in our system.**

# 🔎 MYSTERY #5 — The Trail of the Late Deliveries

**Database:** `Northwinds2022`  
**Theme:** Destinations whose **average planned ship time** is unusually high.

When packages arrive late, you will always hear complaints. After all? who wants to wait for their package in a world where amazon prime exists and shows up at your doorstep in a day or two. One or two late packages is fine but if they arrive consistently late? thats a problem. If its a pattern let's find it.

**Begin Case #5**

### Step 1 — Collect the delivery timeline- we only care about orders already shipped with some timestamp. This shows when an order was placed and when it was shipped.

```sql
SELECT
    o.OrderId,
    o.OrderDate,
    o.ShipToDate,
    o.ShipToCountry
FROM Sales.[Order] AS o
WHERE o.ShipToDate IS NOT NULL
ORDER BY o.OrderId;
```

### Step 2 — Compute days between order and ship dates- here we are just measuring delays and a shipping delay is just time.

```sql
SELECT
    o.OrderId,
    o.OrderDate,
    o.ShipToDate,
    o.ShipToCountry,
    DATEDIFF(day, o.OrderDate, o.ShipToDate) AS PlannedNumberShipDays
FROM Sales.[Order] AS o
WHERE o.ShipToDate IS NOT NULL
ORDER BY PlannedNumberShipDays DESC, o.OrderId;
```

### Step 3 — Lets find the countries with slow shipping ( on average greater than or equal to 8 days). If a single order to ireland is slow? thats life but if its always slow? thats a systemic shipping issue there. Now we see who waits the longest

```sql
SELECT
    o.ShipToCountry,
    AVG(DATEDIFF(day, o.OrderDate, o.ShipToDate)) AS AvgPlannedShipDays,
    COUNT(*) AS OrdersCount
FROM Sales.[Order] AS o
WHERE o.ShipToDate IS NOT NULL
GROUP BY o.ShipToCountry
ORDER BY AvgPlannedShipDays DESC, OrdersCount DESC;
```

### Step 4 — Flag destinations with long averages ( ≥ 8 days) - this is structurally slow, and these countries may be victims of bad trading routes or bad policy but it seems like no one corrected this issue. Ireland, Sweden and United States. The reasons dont matter but the outcomes are the same. Why didn't anyone try to fix this??

```sql
SELECT
    o.ShipToCountry,
    AVG(DATEDIFF(day, o.OrderDate, o.ShipToDate)) AS AvgPlannedShipDays,
    COUNT(*) AS OrdersCount
FROM Sales.[Order] AS o
WHERE o.ShipToDate IS NOT NULL
GROUP BY o.ShipToCountry
HAVING AVG(DATEDIFF(day, o.OrderDate, o.ShipToDate)) >= 8
ORDER BY AvgPlannedShipDays DESC, OrdersCount DESC, o.ShipToCountry;
```

##  **End Case #5 Ireland, Sweden, USA** we have a problem.

# 🔎 MYSTERY #6 — The Supplier’s Favorite 

**Database:** `Northwinds2022`  
**Theme:** Employees funneling orders to **just one** shipper (supplier).

Case Brief: some secrets are written in patterns. Northwinds Purchasing claims every supplier gets a fair shot with equl opportunity. But there is a rumor going around that someone in the purchasing department has a **favorite supplier.**  They might be funneling orders their way over others. We have 1 job, prove or disprove any favoritisim.

**Begin Case #6**

### Step 1 — Inspect employees and their shippers- let's have a look at who orders from who. Every single order has 2 key fingerprints. The one who placed it and who fulfilled that order. We can now see the flow from emloyee to supplier.

```sql
SELECT
    o.OrderId,
    o.EmployeeId,
    o.ShipperId AS SupplierID
FROM Sales.[Order] AS o
ORDER BY o.EmployeeId, SupplierID;
```

### Step 2 — Count orders per (Employee, Supplier)- Where behavior stops being random. If an employee sends 50 orders to supplier A and 0 to everyone else... we have some pattern with motive here/

```sql
SELECT
    o.EmployeeId,
    o.ShipperId AS SupplierID,
    COUNT(*) AS OrdersPlaced
FROM Sales.[Order] AS o
GROUP BY o.EmployeeId, o.ShipperId
ORDER BY o.EmployeeId, OrdersPlaced DESC;
```

### Step 3 — Who uses only one supplier? This is where our mystery cracks open and any Employees with a DistinctSupplierUsed = 1 are the suspects..

```sql
SELECT
    o.EmployeeId,
    COUNT(DISTINCT o.ShipperId) AS DistinctSuppliersUsed,
    COUNT(*) AS TotalOrders
FROM Sales.[Order] AS o
GROUP BY o.EmployeeId
HAVING COUNT(DISTINCT o.ShipperId) = 1
ORDER BY TotalOrders DESC;
```

## **End Case #6 this query returned empty- the rumors were just rumors.**

# 🔎 MYSTERY #7 — The Silent Customers (Never Placed an Order)

  

**Database:** `Northwinds2022`  
**Theme:** Find customers with **no** matching orders.

Case Brief: Silent customers remain a mystery/ Northwinds has a full directory of Names, Companies, Addresses, and Phone numbers. But not every customer actually does business with us. In business, a customer who never orders is a lead that went cold, lost revenue, and a record that shouldnt be in the system at all. We are about to separate our buyers from our ghosts.

**Begin Case #7**

### Step 1 — Peek both sides- before we filter anything at all, let's look at our customer book with everyone we sell to. Customer lists and orders dont always overlap.

```sql
SELECT TOP (10) *
FROM Sales.Customer
ORDER BY CustomerId;

SELECT TOP (10) *
FROM Sales.[Order]
ORDER BY OrderId;
```

### Step 2 — Left join customers to orders- why left join? to keep all customers, even if they never ordered and this will show us customers who have orders and customers who do not.

```sql
SELECT
    c.CustomerId,
    c.CustomerCompanyName,
    o.OrderId
FROM Sales.Customer AS c
LEFT JOIN Sales.[Order] AS o
  ON o.CustomerId = c.CustomerId
ORDER BY c.CustomerId;
```

### Step 3 — Filter to silent customers- there are customers whp exisr in the directory but have never bought a single product.. why were they entered? were they forgotten by our sales team? or do they not like our products? there must be a reason. Regardless of the reason, their silence has meaning to us.

```sql
SELECT
    c.CustomerId,
    c.CustomerCompanyName,
    c.CustomerContactName,
    c.CustomerCountry
FROM Sales.Customer AS c
LEFT JOIN Sales.[Order] AS o
  ON o.CustomerId = c.CustomerId
WHERE o.OrderId IS NULL
ORDER BY c.CustomerCountry, c.CustomerCompanyName;
```

### Step 4 — Add phone numbers for outreach- how about we reach out to them? Let's turn this into a business opportunity and win them back!!

```sql
SELECT
    c.CustomerId,
    c.CustomerCompanyName,
    c.CustomerContactName,
    c.CustomerCountry,
    c.CustomerPhoneNumber
FROM Sales.Customer AS c
LEFT JOIN Sales.[Order] AS o
  ON o.CustomerId = c.CustomerId
WHERE o.OrderId IS NULL
ORDER BY c.CustomerCountry, c.CustomerCompanyName;
```

## **End Case #7- our suspects have been found! Bjorn Tollevsen and Jim Daly. Northwinds would love to have your business in the future.**

## What the data whispered

- We tracked **discount leaks**, **phantom big orders**, **slow lanes**, **supplier bias**, and **silent customers**.
- Each case used **basic SQL**: `SELECT`, `JOIN`, `GROUP BY`, `HAVING`, `ORDER BY`, and a bit of `EXCEPT`.
- The story continues wherever the data leads.

**Thanks for playing SQL Noir.**

**NOTE: Mystery 1 had the help of CHATGPT (LLM) to create the tables and place them into a custom database**

**Mystery 3, 5 and 7 were also done with aid of CHATGPT when i got stuck on a particular solution.**